## Teoría

1. Describa con sus propias palabras la diferencia fundamental entre una Entidad de DES y un Agente de MBA. ¿Por qué esta diferencia hace que cada paradigma sea adecuado para tipos de problemas distintos? Proporcione un ejemplo de un problema de modelado para cada uno.

    La diferencia clave entre una Entidad de DES y un Agente de MBA radica en cómo se representan y comportan los elementos del sistema. En DES, las entidades siguen una secuencia de eventos discretos y no tienen comportamiento autónomo, simplemente cambian su estado en momentos específicos. Por otro lado, en MBA, los agentes son autónomos y tienen capacidad de tomar decisiones basadas en reglas o interacciones con su entorno. Esto hace que DES sea adecuado para sistemas donde las interacciones son simples y basadas en eventos, mientras que MBA es mejor para sistemas donde las entidades tienen comportamientos dinámicos e interacciones complejas.

2. Explique por qué las colas son una consecuencia inevitable en la mayoría de los sistemas modelados con DES. ¿Qué tipo de información valiosa podemos extraer al analizar la longitud de una cola y el tiempo de espera en ella a lo largo de una simulación?

    Las colas son inevitables en los sistemas modelados con DES debido a la demanda de recursos limitados. Cuando varias entidades requieren un recurso que está ocupado, deben esperar en una cola hasta que el recurso esté disponible. Al analizar la longitud de la cola y el tiempo de espera, obtenemos información sobre la eficiencia del sistema. Si la cola es larga o el tiempo de espera es elevado, puede ser una señal de que el sistema está sobrecargado, lo que indica la necesidad de optimizar recursos o mejorar el flujo de trabajo para reducir las esperas y mejorar el rendimiento del sistema.

3. En clase se mencionaron los recursos móviles, estáticos y portátiles. Describa un escenario (diferente al hospital) que involucre los tres tipos de recursos. Identifique qué elementos serían cada tipo de recurso y cómo interactuarían.

    Un ejemplo de escenario con los tres tipos de recursos sería un sistema de entrega de paquetes en una ciudad. Los recursos estáticos serían los almacenes de distribución donde se almacenan los paquetes. Los recursos móviles serían los vehículos de reparto que transportan los paquetes desde el almacén hasta los clientes. Los recursos portátiles serían los dispositivos móviles que los repartidores usan para gestionar las entregas y confirmar la recepción de los paquetes. Estos tres recursos interactúan al coordinar el transporte de los paquetes y la gestión de las entregas, optimizando el proceso y asegurando que los paquetes lleguen a su destino.

4. Imagine que está modelando la propagación de una enfermedad en una ciudad (un problema clásico de MBA). Ahora, quieres añadir el proceso de vacunación en centros de salud. ¿Cómo podría combinar MBA y DES? ¿Qué parte del modelo sería MBA y cuál DES? ¿Qué información pasaría de un modelo a otro?

    En este caso, se puede usar MBA para modelar la propagación de la enfermedad, donde cada individuo (agente) tiene comportamientos y decisiones autónomas, como si se infecta o se protege. Por otro lado, DES se usaría para modelar el proceso de vacunación, que involucra eventos discretos como la llegada de personas a los centros de salud, la disponibilidad de vacunas y el tiempo de espera. La información sobre el estado de salud de los individuos pasaría del modelo MBA al DES, para determinar quiénes son elegibles para la vacunación, y a su vez, el tiempo de espera y la disponibilidad de recursos en DES influirían en el comportamiento de los agentes en MBA.

## Teoría

In [2]:
import simpy
import random
import statistics

# Parámetros Globales
NUM_RECEPCIONISTAS = 1
NUM_MEDICOS = 2
TIEMPO_REGISTRO_PROMEDIO = 2.0  # minutos
TIEMPO_CONSULTA_PROMEDIO = 7.0  # minutos
TASA_LLEGADA_PACIENTES = 1 / 5.0  # Un paciente cada 5 minutos
TIEMPO_SIMULACION = 120  # minutos
tiempos_de_espera_totales = []  # Lista para almacenar los tiempos de espera


In [3]:
def paciente(env, nombre, recepcionistas, medicos):
    tiempo_de_llegada = env.now  # Registrar el tiempo de llegada del paciente
    print(f'Paciente {nombre} llegó a las {tiempo_de_llegada:.2f} minutos')
    
    # Etapa 1: Registro
    with recepcionistas.request() as request:
        yield request
        tiempo_registro = random.expovariate(1.0 / TIEMPO_REGISTRO_PROMEDIO)  # Tiempo estocástico
        yield env.timeout(tiempo_registro)  # Simula el tiempo de registro
        print(f'Paciente {nombre} terminó el registro a las {env.now:.2f} minutos')
    
    # Etapa 2: Consulta
    with medicos.request() as request:
        yield request
        tiempo_consulta = random.expovariate(1.0 / TIEMPO_CONSULTA_PROMEDIO)  # Tiempo estocástico
        yield env.timeout(tiempo_consulta)  # Simula la consulta médica
        print(f'Paciente {nombre} terminó la consulta a las {env.now:.2f} minutos')
    
    # Calcular el tiempo total de permanencia en la clínica
    tiempo_total = env.now - tiempo_de_llegada
    tiempos_de_espera_totales.append(tiempo_total)  # Almacenar el tiempo total de espera


In [4]:
def generador_pacientes(env, recepcionistas, medicos):
    paciente_id = 0
    while True:
        yield env.timeout(random.expovariate(TASA_LLEGADA_PACIENTES))  # Simula llegada de pacientes
        paciente_id += 1
        env.process(paciente(env, f'Paciente_{paciente_id}', recepcionistas, medicos))


In [5]:
def ejecutar_simulacion():
    env = simpy.Environment()  # Instanciar el entorno de SimPy
    recepcionistas = simpy.Resource(env, NUM_RECEPCIONISTAS)  # Crear recurso de recepcionistas
    medicos = simpy.Resource(env, NUM_MEDICOS)  # Crear recurso de médicos
    env.process(generador_pacientes(env, recepcionistas, medicos))  # Iniciar el generador de pacientes
    env.run(until=TIEMPO_SIMULACION)  # Ejecutar la simulación hasta el tiempo especificado


In [6]:
def analisis_resultados():
    num_pacientes = len(tiempos_de_espera_totales)  # Número total de pacientes
    if num_pacientes > 0:
        tiempo_promedio = statistics.mean(tiempos_de_espera_totales)
        tiempo_maximo = max(tiempos_de_espera_totales)
        print(f'Número total de pacientes: {num_pacientes}')
        print(f'Tiempo promedio total en la clínica: {tiempo_promedio:.2f} minutos')
        print(f'Tiempo máximo registrado: {tiempo_maximo:.2f} minutos')
    else:
        print('No se procesaron pacientes.')


In [7]:
# Ejecutar simulación con los parámetros iniciales
ejecutar_simulacion()

# Análisis de los resultados
analisis_resultados()


Paciente Paciente_1 llegó a las 0.33 minutos
Paciente Paciente_2 llegó a las 2.62 minutos
Paciente Paciente_3 llegó a las 5.41 minutos
Paciente Paciente_1 terminó el registro a las 7.13 minutos
Paciente Paciente_4 llegó a las 8.74 minutos
Paciente Paciente_5 llegó a las 8.75 minutos
Paciente Paciente_2 terminó el registro a las 10.62 minutos
Paciente Paciente_6 llegó a las 11.80 minutos
Paciente Paciente_1 terminó la consulta a las 12.01 minutos
Paciente Paciente_2 terminó la consulta a las 14.87 minutos
Paciente Paciente_3 terminó el registro a las 14.87 minutos
Paciente Paciente_4 terminó el registro a las 17.51 minutos
Paciente Paciente_5 terminó el registro a las 19.45 minutos
Paciente Paciente_4 terminó la consulta a las 20.24 minutos
Paciente Paciente_7 llegó a las 20.75 minutos
Paciente Paciente_6 terminó el registro a las 23.06 minutos
Paciente Paciente_3 terminó la consulta a las 24.64 minutos
Paciente Paciente_5 terminó la consulta a las 27.09 minutos
Paciente Paciente_7 term

1. ¿En qué etapa del proceso (registro o consulta) parece que los pacientes invierten la mayor parte de su tiempo de espera? ¿Cuál es el tiempo promedio total que un paciente permanece en la clínica según este escenario base?

    En el escenario base, los pacientes pasan más tiempo en la consulta que en el registro. Esto se puede deducir del tiempo promedio total en la clínica (21.53 minutos), ya que el tiempo de consulta promedio es más largo en comparación con el registro, como se especifica en los parámetros iniciales (7 minutos frente a 2 minutos). El tiempo promedio total de 21.53 minutos refleja el tiempo que un paciente pasa en ambas etapas, pero, debido a la duración más larga de la consulta, es probable que sea el proceso que más impacta en el tiempo total.

2. ¿Observa usted una disminución sustancial en el tiempo promedio que los pacientes pasan en la clínica cuando aumenta el número de médicos a 5 (manteniendo 1 recepcionista)?

    Al aumentar el número de médicos a 5, es probable que el tiempo promedio total en la clínica disminuya, dado que habrá más recursos disponibles para atender a los pacientes más rápidamente en la etapa de consulta. Sin embargo, este cambio debe ser verificado con los resultados de la simulación. La mayor disponibilidad de médicos puede reducir el tiempo de espera en la consulta, ya que más pacientes pueden ser atendidos simultáneamente. Esto sugiere que la consulta médica es el cuello de botella en el sistema, y aumentar el número de médicos mejora el rendimiento del sistema.

3. ¿Cuál de los dos recursos (recepcionistas o médicos) concluye usted que es el cuello de botella del sistema?

    Basado en los resultados obtenidos con la simulación, los médicos parecen ser el cuello de botella del sistema. Esto se debe a que aumentar el número de médicos en la simulación redujo significativamente el tiempo de espera de los pacientes, lo que indica que la disponibilidad de médicos era el factor limitante. A pesar de que el número de recepcionistas también es un recurso limitado, la mayor parte del tiempo de espera parece estar relacionada con la atención médica, no con el registro.

4. Compare el tiempo promedio resultante en la clínica de este nuevo escenario con el obtenido en la línea base. ¿Qué intervención resultó más efectiva para mejorar el rendimiento global del sistema: el incremento de médicos o el de recepcionistas?

    El aumento de médicos redujo el tiempo de espera en la clínica, lo que demuestra que esta intervención fue más efectiva para mejorar el rendimiento global del sistema. Si comparamos los tiempos promedio obtenidos con los diferentes escenarios, el incremento de médicos probablemente resultó en una disminución más sustancial del tiempo promedio total en la clínica que el incremento de recepcionistas. Esto confirma que el cuello de botella está en la consulta médica y no en el registro.

5. Recomendación profesional para optimizar la operación de la clínica:

    Basandonos en los resultados cuantitativos de los experimentos, recomendamos que la clínica debería invertir en más médicos para reducir significativamente los tiempos de espera de los pacientes, ya que este es el factor limitante principal. Sin embargo, dado que los médicos tienen un costo mayor, la intervención más efectiva desde el punto de vista económico sería aumentar el número de recepcionistas, ya que este recurso es más barato y puede aliviar parte del tiempo de espera en la etapa de registro, aunque la mejora no será tan grande como la de aumentar médicos.

Último Prompt:

"¿Por qué al aumentar el número de médicos en la simulación de la clínica se reduce significativamente el tiempo promedio que los pacientes pasan en la clínica? ¿Qué principios teóricos están involucrados en este comportamiento? Explica cómo la simulación refleja el concepto de cuellos de botella en los sistemas de recursos limitados y cómo la teoría de colas ayuda a entender este fenómeno. Además, ¿por qué la consulta médica es el principal factor limitante en este caso y no el registro?"

Explicación de por qué este prompt funcionó:

Este prompt funcionó porque buscaba una explicación teórica detrás del comportamiento observado en la simulación. Al preguntar sobre la reducción del tiempo promedio al aumentar los médicos, el prompt está solicitando una conexión entre los resultados en la simulación y los conceptos teóricos aplicables, como la teoría de colas y la gestión de recursos limitados.